In [1]:
#IMPORTS
import numpy as np 
from matplotlib import pyplot as plt
import torch
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from skimage.util import random_noise
from skimage.filters import gaussian
from skimage.io import imread, imsave
from torchvision.utils import make_grid
import imageio
from IPython.display import Image as Im
import os
from PIL import Image
from google.colab import drive
import scipy.io as sio


# Mount Google Drive
drive.mount('/content/drive')

#GLOBAL VARIABLES (paths)
workspace_path = "/content/drive/My Drive/DeepLearning_2021/Project/"
dataset_path = "arcDataset/"
house_path = "house/"
houseBigData_path = "house_bigdata/"
building_old_path = "building_old/"
building_normal_path = "building_normal/"
building_modern_path = "building_modern/"
church_historical_path = "church_historical/"
results_path = "results/"

Mounted at /content/drive


In [ ]:
selected_style = house_path  # Which of our classes are we going to work with?

In [ ]:
#MAKING OUR OWN CLASS LOADER
class BuildingsDB(torch.utils.data.Dataset):
    # Initialization method for the dataset
    def __init__(self,dataDir = workspace_path+dataset_path+selected_style+'data.mat', transform = None):
        mat_loaded = sio.loadmat(dataDir)
        self.data = mat_loaded["data"]
        self.transform = transform

    # What to do to load a single item in the dataset ( read image. No labels)  
    def __getitem__(self, index):
        data = self.data[index,:,:,:]   
        data = Image.fromarray(data,mode='RGB')
        # Apply a trasnformaiton to the image if it is indicated in the initalizer
        if self.transform is not None : 
            data = self.transform(data)
        
        # return the image and the label
        return data

    # Return the number of images
    def __len__(self):
        return self.data.shape[0]

In [ ]:
#INITIALIZING OUR TRAIN DATA LOADER

# transformation: to tensor
tr = transforms.Compose([
        transforms.ToTensor(), 
        ])

# database: a ".mat" file for a specific style
database = BuildingsDB(workspace_path+dataset_path+selected_style+"data.mat", transform=tr)

# creating the dataloader from the previous database
data_loader = torch.utils.data.DataLoader(dataset=database,
                                           batch_size=128, 
                                           shuffle=True)

# Mini-batch images loading (just to see if everything is correct)
images = next(iter(data_loader))
for i in range(5):
    image = images[i,:,:,:]
    print(image.shape)
    plt.imshow(image.permute(1,2,0).squeeze().numpy())
    plt.axis('off')
    plt.show()

In [2]:
#DEFINITION FOR OUR LAYERS

# Convolution + BatchNormnalization + ReLU block for the encoder
class ConvBNReLU(nn.Module):
    def __init__(self,in_channels, out_channels, pooling=False):
        super(ConvBNReLU, self).__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,kernel_size=3,
                              padding = 1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.pool = None
        if(pooling):
            self.pool = nn.AvgPool2d(2,2)

    def forward(self,x):
        if(self.pool):
            out = self.pool(x)
        else:
            out = x
        out = self.relu(self.bn(self.conv(out)))   
        return out

#  BatchNormalization + ReLU block + Convolution for the decoder
class BNReLUConv(nn.Module):
    def __init__(self,in_channels, out_channels, pooling=False):
        super(BNReLUConv, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels,out_channels,kernel_size=3,
                              padding = 1)

        self.pool = None
        if(pooling):
            self.pool = nn.UpsamplingNearest2d(scale_factor=2)

    def forward(self,x):
        out = self.relu(self.bn(x))
        if(self.pool):
            out = self.pool(out)
        out = self.conv(out)
        return out

In [3]:
# ENCODER / DECODER DEFINITION
#(will be used on the discriminator/generator respectively)

# 3 Conv-BN-ReLU blocks and fully-connected layer
class Encoder(nn.Module):
    def __init__(self,out_features,base_channels=16):
        super(Encoder, self).__init__()
        self.layer1 = ConvBNReLU(3,base_channels,pooling=False)
        self.layer2 = ConvBNReLU(base_channels,base_channels*2,pooling=True)
        self.layer3 = ConvBNReLU(base_channels*2,base_channels*4,pooling=True)
        self.fc = nn.Linear(32*32*base_channels*4,out_features)
  
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        return self.fc(out.view(x.shape[0],-1))
    
# 3 BN-ReLU-Conv blocks 
class Decoder(nn.Module):
    def __init__(self,out_features,base_channels=16):
        super(Decoder, self).__init__()
        self.base_channels = base_channels
        self.fc = nn.Linear(out_features,32*32*base_channels*4)
        self.layer3 = BNReLUConv(base_channels*4,base_channels*2,pooling=True)
        self.layer2 = BNReLUConv(base_channels*2,base_channels,pooling=True)
        self.layer1 = BNReLUConv(base_channels,3,pooling=False)
  
    def forward(self,x):
        out = self.fc(x)
        out = out.view(x.shape[0],self.base_channels*4,32,32)
        out = self.layer3(out)
        out = self.layer2(out)
        out = self.layer1(out)
        return torch.sigmoid(out)

In [4]:
# DISCRIMINATOR / GENERATOR DEFINITION

# Discriminator similar to VAE encoder
class Discriminator(nn.Module):
    def __init__(self, base_channels=16):
        super(Discriminator, self).__init__()
        # last fully connected layer acts as a a binary classifier
        self.classifier = Encoder(3,base_channels)

    # Forward pass obtaining the discriminator probability
    def forward(self,x):
        out = self.classifier(x)
        # use sigmoid to get the real/fake image probability
        return torch.sigmoid(out)

# Generator is defined as VAE decoder
class Generator(nn.Module):
    def __init__(self,in_features,base_channels=16):
        super(Generator, self).__init__()
        self.base_channels = base_channels
        self.in_features = in_features
        self.decoder = Decoder(in_features,base_channels)

    # Generate an image from vector z
    def forward(self,z):
        return torch.sigmoid(self.decoder(z))

    # Sample a set of images from random vectors z
    def sample(self,n_samples=128,device='cpu'):
        samples_unit_normal = torch.randn((n_samples,self.in_features)).to(device)
        return self.decoder(samples_unit_normal)


In [5]:
# GAN TRAIN FUNCTION
def train_GAN(gen, disc,  train_loader, optimizer_gen, optim_disc,
              num_epochs=10, model_name='gan_architecture.ckpt', device='cpu'):
    gen = gen.to(device)
    gen.train() # Set the generator in train mode
    disc = disc.to(device)
    disc.train() # Set the discriminator in train mode

    total_step = len(train_loader)

    #Initializing arrays to return
    gen_losses_list = []
    disc_losses_list = []
    gen_mse_losses_list = []

    criterion = nn.MSELoss()  # This one will be the MSE loss between fake and real images

    # Iterate over epochs
    for epoch in range(num_epochs):
        
        disc_loss_avg = 0
        gen_loss_avg = 0
        gen_mse_loss_avg = 0

        nBatches = 0
        update_generator = True  # Learning on only one model at each epoch

        # Iterate the dataset
        for i, real_images in enumerate(train_loader):
            # Get batch of samples and labels
            real_images = real_images.to(device)
            n_images = real_images.shape[0]

            # Forward pass
            # Generate random images with the generator
            fake_images = gen.sample(n_images,device=device)

            # Use the discriminator to obtain the probabilties for real and generate imee
            prob_real = disc(real_images)
            prob_fake = disc(fake_images)
            
            # Generator loss
            gen_loss = -torch.log(prob_fake).mean()
            # Discriminator loss
            disc_loss = -0.5*(torch.log(prob_real) + torch.log(1-prob_fake)).mean()
            #MSE Loss fake images vs real images
            gen_mse_loss = criterion(fake_images,real_images)
            
            # We are going to update the discriminator and generator parameters alternatively at each iteration

            if(update_generator):
                # Optimize generator
                # Backward and optimize
                optimizer_gen.zero_grad()
                gen_loss.backward() # Necessary to not erase intermediate variables needed for computing disc_loss gradient
                optimizer_gen.step()
                update_generator = False
            else:        
                # Optimize discriminator
                # Backward and optimize
                optimizer_disc.zero_grad()
                disc_loss.backward()
                optimizer_disc.step()
                update_generator = True
            
            #Updating loss
            disc_loss_avg += disc_loss.cpu().item()
            gen_loss_avg += gen_loss.cpu().item()
            gen_mse_loss_avg += gen_mse_loss.cpu().item()

            nBatches+=1

        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Gen. Loss: {:.4f}, Disc Loss: {:.4f}, Gen. MSE Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, i+1, total_step, gen_loss_avg / nBatches, disc_loss_avg / nBatches, gen_mse_loss_avg / nBatches))
        
        # Save model
        gen_losses_list.append(gen_loss_avg / nBatches)
        disc_losses_list.append(disc_loss_avg / nBatches)
        gen_mse_losses_list.append(gen_mse_loss_avg / nBatches)
        torch.save(gen.state_dict(), workspace_path+results_path+ '/' + model_name)

        #This part will only be executed for every 100 epochs  
        if epoch%100!=0:
            continue
        
        # Creating a grid of 4x4 sample images generated from our generator
        x_gen = gen.sample(n_samples=16,device=device)
        image_grid = make_grid(x_gen.cpu(),nrow=4,padding=1)
        plt.figure(figsize=(8,8))
        plt.imshow(image_grid.permute(1,2,0).detach().numpy())
        plt.show()

        # This part will only be executed at the last epoch
        if epoch != num_epochs-1:
            continue

        # Creating a .gif with 50 interpolations between two Z random vectors 
        # and saving it    
        n_samples = 16
        n_iterpolations =50

        z_init = torch.randn((n_samples,gen.in_features)).to(device)
        z_final = torch.randn((n_samples,gen.in_features)).to(device)

        interpolation_images = []
        for interp in range(0,n_iterpolations):
            interp_0_1 = float(interp) / (n_iterpolations-1)
            z = z_init*interp_0_1 + z_final*(1-interp_0_1)
            x_rec = gen.decoder(z.to(device))
            image_grid = make_grid(x_rec.cpu(),nrow=4,padding=1)
            image_grid = image_grid.permute(1,2,0).detach().numpy()

            interpolation_images.append((image_grid*255.0).astype(np.uint8))
        
        interpolation_images += interpolation_images[::-1]

        imname = workspace_path+results_path+'/gan_interpolation_churches_historical.gif'
        imageio.mimsave(imname, interpolation_images, fps=25)

    return gen_losses_list, disc_losses_list, gen_mse_losses_list

In [ ]:
# TRAINING

# Define Geneartor and Discriminator networks
gan_gen = Generator(32) 
gan_disc = Discriminator()

#Initialize independent optimizer for both networks
learning_rate = .0004
optimizer_gen = torch.optim.Adam(gan_gen.parameters(),lr = learning_rate, betas=(0.2,0.5), weight_decay=1e-5)
learning_rate = .00001
optimizer_disc = torch.optim.Adam(gan_disc.parameters(),lr = learning_rate, betas=(0.2,0.5), weight_decay=1e-5)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Train the GAN
gen_loss_list, disc_loss_list, gen_mse_loss_list = train_GAN(gan_gen,gan_disc, data_loader, optimizer_gen, optimizer_disc,
                      num_epochs=20000, model_name='churches_historical.ckpt', device=device)

#Plotting the loss for each epoch
plt.plot(gen_loss_list, color="r", label="generator")
plt.plot(disc_loss_list, color="b", label="discriminator")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

#Plotting the mse loss for the generator (between real and fake images) for each epoch
plt.plot(gen_mse_loss_list)
plt.xlabel("Epoch")
plt.ylabel("MSE Loss (fake vs. real images)")
plt.show()

In [ ]:
#LOADING A SAVED MODEL AND PLOTTING 4X4 SAMPLE IMAGES

gan_gen = Generator(32)
gan_gen.load_state_dict(torch.load(workspace_path+results_path+"house_20000ep.ckpt",map_location='cuda:0'))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
x_gen = gan_gen.sample(n_samples=16,device="cpu")


image_grid = make_grid(x_gen.cpu(),nrow=4,padding=1)
plt.figure(figsize=(8,8))
plt.imshow(image_grid.permute(1,2,0).detach().numpy())
plt.show()